### Ensembles with pipeline and column transformer

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=sns.load_dataset('tips')

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
# predict what's the time if its lunch or dinner>>time target variabel

df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [5]:
# EDA>> subjective
# encoding, missing valuse treatement , scalling>>>> automate this
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [6]:
# since time is nomianal variable, we will use label encoder
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [7]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2


In [8]:
df.time.unique()  
# dinner >> 1
# lunch >> 0

array([0, 1])

In [9]:
df.isna().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [10]:
X=df.drop('time', axis=1)
y=df['time']

In [11]:
X

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [12]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train ,y_test=train_test_split(X, y, test_size=0.20, random_state=1)

In [14]:
X_train

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
154,19.77,2.00,Male,No,Sun,4
167,31.71,4.50,Male,No,Sun,4
110,14.00,3.00,Male,No,Sat,2
225,16.27,2.50,Female,Yes,Fri,2
...,...,...,...,...,...,...
137,14.15,2.00,Female,No,Thur,2
72,26.86,3.14,Female,Yes,Sat,2
140,17.47,3.50,Female,No,Thur,2
235,10.07,1.25,Male,No,Sat,2


In [15]:
df.isna().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [16]:
# handling the missing value
# data encoding
# feature scalling

# automat

from sklearn.impute import SimpleImputer   # for missing value
from sklearn.preprocessing import OneHotEncoder  # for encoding the data
from sklearn.preprocessing import StandardScaler   # for scalling

from sklearn.pipeline import Pipeline  # a sequence of data tranformer in order>> step1, step2, step3
from sklearn.compose import ColumnTransformer  # group all the pipline steps for each of the colum

In [17]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [18]:
cat_cols=['sex', 'smoker', 'day']
num_cols=['total_bill', 'tip', 'size']

In [19]:
# feature engineering automation using pipeline and columns transformer

num_pipeline=Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),
                             ('scaling', StandardScaler()) ])  # median beause outlier tretment nhi kiye h

cat_pipeline=Pipeline(steps=[('imputation', SimpleImputer(strategy='most_frequent')),
                              ('encoding', OneHotEncoder())]) 

In [20]:
preprocessor=ColumnTransformer([('num_pipeline', num_pipeline, num_cols), ('cat_pipeline', cat_pipeline, cat_cols)])

In [21]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [22]:
X_train

array([[-0.28611937, -1.47443803, -0.57766863, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.02695905, -0.71612531,  1.47042924, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.3716196 ,  1.19880579,  1.47042924, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.23206267,  0.43283335, -0.57766863, ...,  0.        ,
         0.        ,  1.        ],
       [-1.06543688, -1.29060464, -0.57766863, ...,  1.        ,
         0.        ,  0.        ],
       [-0.29287646,  0.1034652 ,  0.44638031, ...,  1.        ,
         0.        ,  0.        ]])

In [23]:
X_test

array([[-1.85376383, -1.48209775, -1.60171757,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.08453291,  0.04984713, -0.57766863,  1.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.79501474,  0.36389583,  0.44638031,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.59356688, -0.33313909, -0.57766863,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.18349826,  0.04984713, -0.57766863,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-1.32783714, -1.14506988, -0.57766863,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.   

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

models={'support vector classifier': SVC(), 
        'DT classifier': DecisionTreeClassifier(),
        'Log Regression': LogisticRegression()}

In [25]:
len(models)

3

In [26]:
models.values()

dict_values([SVC(), DecisionTreeClassifier(), LogisticRegression()])

In [27]:
list(models.values())

[SVC(), DecisionTreeClassifier(), LogisticRegression()]

In [28]:
models.keys()

dict_keys(['support vector classifier', 'DT classifier', 'Log Regression'])

In [29]:
list(models.keys())

['support vector classifier', 'DT classifier', 'Log Regression']

In [30]:
from sklearn.metrics import accuracy_score
def model_train_eval(X_train,y_train, X_test, y_test, models): 
    evaluation={}
    for i in range(len(models)): 
        model=list(models.values())[i]
        model.fit(X_train, y_train)
        y_pred=model.predict(X_test)
        model_score=accuracy_score(y_test, y_pred)
        evaluation[list(models.keys())[i]]=model_score
    return evaluation

In [31]:
model_train_eval(X_train, y_train, X_test, y_test, models)

{'support vector classifier': 0.9183673469387755,
 'DT classifier': 0.8979591836734694,
 'Log Regression': 0.9183673469387755}